In [56]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import streamlit as st

Data Prepration

In [46]:
DATA_PATH = 'C:/streamlit-tiktok/toma-streamlit-tiktok/toma-streamlit-tiktok/csv/'
tiktok_dataset = pd.read_csv(f'{DATA_PATH}bq-results-20230329-085959-1680080406863.csv', encoding='utf-8')

C:\Users\jangs\AppData\Local\Temp\ipykernel_12632\3654686385.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  tiktok_dataset = pd.read_csv(f'{DATA_PATH}bq-results-20230329-085959-1680080406863.csv', encoding='utf-8')


In [47]:
for col in tiktok_dataset.columns:
    n_nan = tiktok_dataset[col].isnull().sum()
    if n_nan>0:
      msg = '{:^20}에서 결측치 개수 : {}개'.format(col,n_nan)
      print(msg)

collected_videos_count에서 결측치 개수 : 6757개
  collection_time   에서 결측치 개수 : 6757개
   influencer_id    에서 결측치 개수 : 6757개
 profile_pic_url_hd 에서 결측치 개수 : 83573개
         id         에서 결측치 개수 : 6757개
      nickname      에서 결측치 개수 : 6757개
     followers      에서 결측치 개수 : 6757개
       hearts       에서 결측치 개수 : 6757개
    videos_count    에서 결측치 개수 : 6757개


In [48]:
dt_tiktok = tiktok_dataset.drop([tiktok_dataset.columns[3]],axis=1)

In [49]:
DF_tiktok1 = DF_tiktok[DF_tiktok['influencer_id'].notna()]
DF_tiktok1.shape

(84969, 8)

In [50]:
for col in DF_tiktok1.columns:
    n_nan = DF_tiktok1[col].isnull().sum()
    if n_nan>0:
      msg = '{:^20}에서 결측치 개수 : {}개'.format(col,n_nan)
      print(msg)
    else:
      print('결측치가 없습니다.')

결측치가 없습니다.
결측치가 없습니다.
결측치가 없습니다.
결측치가 없습니다.
결측치가 없습니다.
결측치가 없습니다.
결측치가 없습니다.
결측치가 없습니다.


In [55]:
DF_tiktok1 = pd.DataFrame(data=DF_tiktok1, index=None, columns=None, dtype=None, copy=False)
DF_tiktok1.shape
DF_tiktok1.columns
print(DF_tiktok1)


       collected_videos_count  collection_time  influencer_id  \
234                       0.0     1.674149e+12   6.790741e+18   
235                       0.0     1.674150e+12   6.541867e+16   
236                       0.0     1.674150e+12   6.656208e+18   
237                       0.0     1.674149e+12   6.696405e+18   
238                       0.0     1.674149e+12   7.103947e+18   
...                       ...              ...            ...   
91721                    10.0     1.678555e+12   6.735398e+18   
91722                    10.0     1.678556e+12   6.786789e+10   
91723                    10.0     1.678555e+12   6.698975e+18   
91724                    10.0     1.678555e+12   6.795390e+18   
91725                    10.0     1.678555e+12   6.740171e+18   

                    id         nickname  followers       hearts  videos_count  
234        linhngadang    Linh Nga Dang       42.0          0.0           0.0  
235         zynzgyniee         Khánh Hà      177.0         

In [60]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import json
import pickle
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot, plot
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import matplotlib
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import ExtraTreesRegressor

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')


In [64]:
df_tiktok_dataset = DF_tiktok1
df_tiktok_dataset.head(5)
df_tiktok_dataset.info()
df_tiktok_dataset.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84969 entries, 234 to 91725
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   collected_videos_count  84969 non-null  float64
 1   collection_time         84969 non-null  float64
 2   influencer_id           84969 non-null  float64
 3   id                      84969 non-null  object 
 4   nickname                84969 non-null  object 
 5   followers               84969 non-null  float64
 6   hearts                  84969 non-null  float64
 7   videos_count            84969 non-null  float64
dtypes: float64(6), object(2)
memory usage: 5.8+ MB


(84969, 8)

In [71]:
data = df_tiktok_dataset[['collected_videos_count','collection_time','influencer_id','id','nickname', 'followers','hearts', 'videos_count']]
data.isnull().sum()

collected_videos_count    0
collection_time           0
influencer_id             0
id                        0
nickname                  0
followers                 0
hearts                    0
videos_count              0
dtype: int64

Modeling and Viz

In [73]:
df_unique_videos = df_tiktok_dataset.drop_duplicates(subset='id', keep="first")

In [76]:
# Focus on dataset from 0 till 50.000 likes
df_videos_users_focus = df_unique_videos[df_unique_videos['hearts'] >= 10]

In [78]:
### get the output df to use in app.py :
# Focus on more than 10 data sets of likes
df_videos_users_focus = df_unique_videos[df_unique_videos['hearts'] >= 10]
df_videos_users_focus.to_csv('df_videos_users_focus.csv',index=False)